[View in Colaboratory](https://colab.research.google.com/github/hamelsmu/kdd-2018-hands-on-tutorials/blob/master/TensorFlow_with_GPU.ipynb)

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import pandas as pd
issues = pd.read_csv('https://storage.googleapis.com/kubeflow-examples/github-issue-summarization-data/github-issues.zip')

# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [4]:
import urllib2

In [9]:
f = urllib2.urlopen('https://storage.googleapis.com/kubeflow-examples/code_search/data/train.function')
source_docs = f.readlines()

In [6]:
source_docs

['def batch_generator batch_size data labels None n_batches int np ceil len data float batch_size idx np random permutation len data data_shuffled data idx if labels is not None labels_shuffled labels idx for i in range n_batches start i batch_size end start batch_size if labels is not None yield data_shuffled start end labels_shuffled start end else yield data_shuffled start end\n',
 'def to_categorical labels num_classes new_labels np zeros len labels num_classes label_to_idx_map idx_to_label_map dict dict idx 0 for i label in enumerate labels if label not in label_to_idx_map label_to_idx_map label idx idx_to_label_map idx label idx 1 new_labels i label_to_idx_map label 1 return new_labels label_to_idx_map idx_to_label_map\n',
 'classmethod def function cls x return 1 1 0 np exp x\n',
 'classmethod def prime cls x return x 1 x\n',
 'classmethod def function cls x return np maximum np zeros x shape x\n',
 'classmethod def prime cls x return x 0 astype int\n',
 'classmethod def functio

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
8.350230318000058
GPU (s):
0.1842791589999706
GPU speedup over CPU: 45x
